<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Processamento de Linguagem Natural com Transformers</font>

## <font color='blue'>Projeto 4</font>
## <font color='blue'>Fine Tuning do LLM Llama 2 Para Geração de Texto</font>

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.10.12


In [2]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark.
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [3]:
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 --user

In [4]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [5]:
# Imports
import os
import torch
import peft
import trl
import datasets
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from datasets import load_dataset
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          HfArgumentParser,
                          TrainingArguments,
                          pipeline,
                          logging)
import warnings
warnings.filterwarnings('ignore')

https://pypi.org/project/peft/

https://pypi.org/project/trl/

https://pypi.org/project/bitsandbytes/

https://pypi.org/project/accelerate/

In [6]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

Author: Data Science Academy

peft        : 0.4.0
trl         : 0.4.7
datasets    : 2.15.0
torch       : 2.1.0+cu118
transformers: 4.31.0



In [7]:
if torch.cuda.is_available():
    print('Número de GPUs:', torch.cuda.device_count())
    print('Modelo GPU:', torch.cuda.get_device_name(0))
    print('Total Memória [GB] da GPU:',torch.cuda.get_device_properties(0).total_memory / 1e9)

Número de GPUs: 1
Modelo GPU: NVIDIA A100-SXM4-40GB
Total Memória [GB] da GPU: 42.481549312


## Definindo o Modelo Llama 2 Base

In [8]:
# Nome do modelo
modelo_base = "NousResearch/Llama-2-7b-chat-hf"

## Definindo o Dataset Para o Ajuste Fino

https://huggingface.co/datasets/mlabonne/guanaco-llama2-1k

In [9]:
# Dataset usado para o ajuste fino
nome_dataset = "mlabonne/guanaco-llama2-1k"

## Definindo Parâmetros LoRA

https://huggingface.co/docs/peft/conceptual_guides/lora

In [10]:
# Dimensão da atenção LoRA
lora_r = 64

In [11]:
# Parâmetro Alpha para LoRA scaling
lora_alpha = 16

In [12]:
# Probabilidade Dropout para as camadas LoRA
lora_dropout = 0.1

## Definindo Parâmetros bitsandbytes

In [13]:
# Ativa precisão 4-bits ao carregar o modelo base
use_4bit = True

In [14]:
# Define o dtype para os parâmetros do modelo base
bnb_4bit_compute_dtype = "float16"

Quantização: É um processo de redução da precisão dos números usados em um modelo de Machine Learning. Por exemplo, em vez de usar números de ponto flutuante de 32 bits (como FP32), a quantização pode reduzir esses números para representações de 16 bits, 8 bits, ou até menos. O objetivo é diminuir o tamanho do modelo e aumentar a velocidade de inferência, mantendo ao máximo a precisão do modelo.

https://arxiv.org/abs/2208.07339

In [15]:
# Tipo de quantização (fp4 ou nf4)
# Podemos usar o tipo NF4, que é um tipo de dados de 4 bits adaptado para pesos que foram inicializados usando uma distribuição normal.
# NF4 = Normal Float 4
bnb_4bit_quant_type = "nf4"

In [16]:
# Desativa a quantização aninhada para modelos básicos de 4 bits (quantização dupla)
use_nested_quant = False

## Definindo Parâmetros de Treinamento

In [17]:
# Diretório de saída onde as previsões e os checkpoints do modelo serão armazenados
output_dir = "resultados"

In [18]:
# Número de épocas de treino
num_train_epochs = 2

https://en.wikipedia.org/wiki/Bfloat16_floating-point_format

In [19]:
# Habilita o treinamento fp16/bf16 (defina bf16 como True com um A100)
fp16 = False
bf16 = True

In [20]:
# Batch size por GPU para treino
per_device_train_batch_size = 4

In [21]:
# Batch size por GPU para avaliação
per_device_eval_batch_size = 4

In [22]:
# Número de etapas de atualização para acumular os gradientes
gradient_accumulation_steps = 1

In [23]:
# Habilita gradient checkpointing
gradient_checkpointing = True

In [24]:
# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

In [25]:
# Taxa de aprendizado
learning_rate = 2e-4

In [26]:
# Redução de peso a ser aplicada a todas as camadas, exceto pesos bias/LayerNorm
weight_decay = 0.001

In [27]:
# Otimizador
optim = "paged_adamw_32bit"

Ao usar lr_scheduler_type = "cosine", está-se especificando que o agendador de taxa de aprendizagem deve seguir esta estratégia de cosseno para ajustar a taxa de aprendizagem durante o treinamento do modelo. Essa abordagem é muitas vezes eficaz para obter uma convergência mais rápida e estável do modelo.

In [28]:
# Learning rate schedule
lr_scheduler_type = "cosine"

In [29]:
# Número de etapas de treinamento
max_steps = -1

O warmup_ratio especifica a proporção do treinamento total que será dedicada ao aquecimento. Por exemplo, um warmup_ratio de 0.03 significa que 3% do treinamento total será usado para aquecer a taxa de aprendizagem. Se o treinamento total for de 100 épocas, por exemplo, o aquecimento ocorrerá nas primeiras 3 épocas.

Durante a fase de aquecimento, a taxa de aprendizagem aumenta linearmente de um valor inicial (geralmente próximo de zero) até o valor de taxa de aprendizagem estabelecido para o treinamento regular. Após o período de aquecimento, a taxa de aprendizagem segue o plano estabelecido, que pode ser constante, decrescente ou seguindo algum outro agendador.

In [30]:
# Proporção de passos para um aquecimento linear (de 0 à taxa de aprendizagem)
warmup_ratio = 0.03

In [31]:
# Agrupar sequências em lotes com o mesmo comprimento
# Economiza memória e acelera consideravelmente o treinamento
group_by_length = True

In [32]:
# Salvar checkpoint a cada X etapas de atualização
save_steps = 0

In [33]:
# Registrar cada X etapas de atualizações
logging_steps = 25

## Definindo Parâmetros do Ajuste Fino

https://huggingface.co/docs/trl/sft_trainer

In [34]:
# Comprimento máximo de sequência a ser usado
max_seq_length = None

In [35]:
# Reunir vários exemplos curtos na mesma sequência de entrada para aumentar a eficiência
packing = False

In [36]:
# Carregar o modelo inteiro na GPU 0
device_map = {"": 0}

In [37]:
# Carrega o dataset com a amostra de treino
dataset = load_dataset(nome_dataset, split = "train")

In [38]:
# Carrega o tokenizer e modelo com configuração QLoRA
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

In [39]:
bnb_config = BitsAndBytesConfig(load_in_4bit = use_4bit,
                                bnb_4bit_quant_type = bnb_4bit_quant_type,
                                bnb_4bit_compute_dtype = compute_dtype,
                                bnb_4bit_use_double_quant = use_nested_quant)

In [40]:
# Verificando se a GPU é compatível com bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("A GPU suporta bfloat16: acelere o processo de treino com bf16=True")
        print("=" * 80)

A GPU suporta bfloat16: acelere o processo de treino com bf16=True


## Carregando o Modelo Base e o Tokenizador

In [41]:
# Carrega o modelo base
modelo_dsa = AutoModelForCausalLM.from_pretrained(modelo_base,
                                                  quantization_config = bnb_config,
                                                  device_map = device_map)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [42]:
modelo_dsa.config.use_cache = False

In [43]:
modelo_dsa.config.pretraining_tp = 1

In [44]:
# Carrega o tokenizador do LLaMA2
tokenizer = AutoTokenizer.from_pretrained(modelo_base, trust_remote_code = True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

## Carregando a Configuração LoRA e SFTTrainer

In [45]:
# Carrega a configuração LoRA
peft_config = LoraConfig(lora_alpha = lora_alpha,
                         lora_dropout = lora_dropout,
                         r = lora_r,
                         bias = "none",
                         task_type = "CAUSAL_LM")

In [46]:
# Parâmetros de configuração de treino
training_arguments = TrainingArguments(output_dir = output_dir,
                                       num_train_epochs = num_train_epochs,
                                       per_device_train_batch_size = per_device_train_batch_size,
                                       gradient_accumulation_steps = gradient_accumulation_steps,
                                       optim = optim,
                                       save_steps = save_steps,
                                       logging_steps = logging_steps,
                                       learning_rate = learning_rate,
                                       weight_decay = weight_decay,
                                       fp16 = fp16,
                                       bf16 = bf16,
                                       max_grad_norm = max_grad_norm,
                                       max_steps = max_steps,
                                       warmup_ratio = warmup_ratio,
                                       group_by_length = group_by_length,
                                       lr_scheduler_type = lr_scheduler_type,
                                       report_to = "tensorboard")

In [47]:
# Definir parâmetros de ajuste fino com SFTTrainer
trainer = SFTTrainer(model = modelo_dsa,
                     train_dataset = dataset,
                     peft_config = peft_config,
                     dataset_text_field = "text",
                     max_seq_length = max_seq_length,
                     tokenizer = tokenizer,
                     args = training_arguments,
                     packing = packing)

## Treinamento e Ajuste Fino

In [48]:
# Treina o modelo
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


TrainOutput(global_step=500, training_loss=1.3034837684631349, metrics={'train_runtime': 474.4702, 'train_samples_per_second': 4.215, 'train_steps_per_second': 1.054, 'total_flos': 1.751271512702976e+16, 'train_loss': 1.3034837684631349, 'epoch': 2.0})

In [49]:
# Salva o modelo ajustado
trainer.model.save_pretrained("llama-2-7b-ajustado")

## Gerando Texto com o Modelo Ajustado

In [50]:
# Ignora warnings
logging.set_verbosity(logging.CRITICAL)

In [51]:
# Prepara o prompt
prompt = "O que são grandes modelos de linguagem (LLMs)?"

In [52]:
# Cria o pipeline
pipe = pipeline(task = "text-generation",
                model = modelo_dsa,
                tokenizer = tokenizer,
                max_length = 200)

In [53]:
# Executa o pipeline e gera o texto a partir do prompt inicial
resultado = pipe(f"<s>[INST] {prompt} [/INST]")

In [54]:
print(resultado[0]['generated_text'])

<s>[INST] O que são grandes modelos de linguagem (LLMs)? [/INST] Grandes modelos de linguagem (LLMs) são modelos de aprendizado de máquina que são treinados com grandes quantidades de dados e são capazes de realizar tarefas complexas, como responder perguntas, traduzir textos, criar textos e realizar tarefas de escrita.

Esses modelos são geralmente usados em aplicativos de inteligência artificial, como chatbots, assistentes virtuais e sistemas de recomendação. Eles são capazes de aprender a partir dos dados que são fornecidos a eles e podem realizar tarefas de análise de texto, reconhecimento de fala e reconhecimento de imagem.

Grandes model


# Fim